In [3]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [4]:
def pretty_print_docs(docs):
    for i, d in enumerate(docs):
        print(f"문서 {i+1}:\n\n" + d.page_content)
        print(f"{'-' * 100}\n")

In [5]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

In [6]:
from langchain_openai import ChatOpenAI

### **EmbeddingsFilter**

`EmbeddingsFilter`는 문서와 쿼리를 임베딩하고 쿼리와 충분히 유사한 임베딩을 가진 문서만 반환함으로써 더 저렴하고 빠른 옵션을 제공합니다.<br>

`EmbeddingsFilter`는 `embedding` 기반으로 필터합니다. <br>
embedding 기반으로 필터를 할때에 유사도를 다시 한번 계산해서 similarity_threshold 값을 주고 그 값(threshold) 이상인 것들만 필터링을 걸게 됩니다. <br>

In [7]:
loader = TextLoader("./data/appendix-keywords.txt", encoding='utf-8')

In [8]:
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
texts = loader.load_and_split(text_splitter)

In [9]:
embeddings = OpenAIEmbeddings(api_key=key)

In [10]:
# 기본 retriever 생성
retriever = FAISS.from_documents(texts, embedding=embeddings).as_retriever()

In [11]:
llm = ChatOpenAI(
    api_key=key, 
    model_name='gpt-4o-mini',
    temperature=0.1
)

In [12]:
# 유사도 임계값이 0.86인 EmbeddingsFilter 객체를 생성
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.86)

In [13]:
# ContextualCompressionRetriever 객체를 생성
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter,  # 기본 압축기로 embeddings_filter를 지정
    base_retriever=retriever            # 기본 검색기로 retriever 지정
)

In [14]:
# ContextualCompressionRetriever 객체를 사용하여 관련 문서를 검색합니다.
compressed_docs = compression_retriever.invoke(
    "Semantic Search 에 대해서 알려줘."
)

In [15]:
pretty_print_docs(compressed_docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding
----------------------------------------------------------------------------------------------------

